In [1]:
import PyVal
import numpy as np
import pickle
import matplotlib.pyplot as plt
import ipyparallel as ipp
from functools import reduce

In [2]:
%matplotlib
#%px import ipyparallel as ipp; ipp.bind_kernel()
#%px %pylab inline
#%px %qtconsole
#%px %connect_info
rc = ipp.Client()
rc.ids
with rc[:].sync_imports():
    import PyVal
    import numpy as np
    import pickle
    import matplotlib.pyplot as plt
dview = rc.load_balanced_view()

Using matplotlib backend: TkAgg
importing PyVal on engine(s)
importing numpy on engine(s)
importing pickle on engine(s)
importing matplotlib.pyplot on engine(s)


/home/julian/.local/lib/python3.6/site-packages/ipyparallel/client/client.py:459: RuntimeWarning: 
            Controller appears to be listening on localhost, but not on this machine.
            If this is true, you should specify Client(...,sshserver='you@julian-ThinkPad-W530')
            or instruct your controller to listen on an external IP.
  RuntimeWarning)


In [37]:
#@ipp.require('PyVal', 'numpy')
def run_sim(N, val, pList, T, r):
    import numpy as np
    nw = PyVal.BS_Network()
    MList = []
    rsList= []
    assetList= []
    deltaList = []
    solventList = []
    MVarList = []
    rsVarList= []
    assetVarList= []
    deltaVarList = []
    solventVarList = []
    #vOutList = []
    for p in pList:
        print("Runing N=" +str(N)+ ", val="+str(val)+", p="+str(p))
        nw.run(N, p, val, 2, T, r, 2000, 200)
        MList.append(nw.get_M())
        rsList.append(np.array(nw.get_rs())[:,0])
        assetList.append(np.array(nw.get_assets())[:,0])
        solventList.append(np.array(nw.get_solvent())[:,0])
        deltaList.append(nw.get_delta_jacobians())
        MVarList.append(nw.get_M_var())
        rsVarList.append(np.array(nw.get_rs_var())[:,0])
        assetVarList.append(np.array(nw.get_assets_var())[:,0])
        solventVarList.append(np.array(nw.get_solvent_var())[:,0])
        deltaVarList.append(nw.get_delta_jacobians_var())
    res = {'N': N, 'conn': (N*np.array(pList)).tolist(), 'T':T, 'r': r ,'p': pList, 'val': val, 'M': MList, 'M var': MVarList,\
           'Assets': assetList, 'Assets var': assetVarList,\
		'RS': rsList, 'Delta': deltaList, 'Solvent': solventList, 'RS var': rsVarList, 'Delta var': deltaVarList, 'Solvent var': solventVarList}
    return res

# callapses a list of results into a single result.
# returns a tuple: (Collapsed result, list of results with wrong parameters)
# TODO: allow for collaps along, r, T, val, N. for now only p
# TODO: sort along collapse axis
def combine_results(*results):
    r0 = results[0]
    res_new = r0.copy()
    failed = []
    if type(r0) is not dict:
        print("WARNING! Did recieve non-dict type as parameter")
    

    def combine(el):
        if r0['val'] != el['val'] or r0['N'] != el['N']\
        or r0['T'] != el['T'] or r0['r'] != el['r']:
            return False
        for key, value in el.items():
            if key == 'val' or key == 'N' or key == 'T' or key == 'r':
                continue
            if type(el[key]) is not list:
                print("WARNING! Key: " + key + " is not a list")
            res_new[key] = res_new[key] + el[key]
        return True
            
    for el in results[1:]:
        if type(el) is not dict:
            print("WARNING! Did recieve non-dict type as parameter")
        if not combine(el):
            failed.append(el)
            
    return (res_new, failed)

def test1(n):
    r = 0
    for i in range(n):
        r += i
    return r

def pList(N, pts):
    import numpy as np
    return np.union1d(np.linspace(0.0,1.8/N,pts), np.linspace(0.7/N,1.3/N,pts))

# Parameters
## Black Scholes
    - T = 1
    - r = 0
    
## Network Parameters
    - N = [15, 25, 50, 75, 100, 150, 200, 300]
    - <k> = n*p = [0 .. 1.8]
    - M => ER(N,p)

In [35]:
dview.block=False
in_0_1 = [(N,0.1, [el], 1.0, 0.0) for N in [3, 5] for el in pList(N,3)]#, (25,0.1,pList(25,8)), (50,0.1,pList(50,6)), (75, 0.1, pList(75,6))]#, (100, 0.1, pList(100,4)), (200, 0.1, pList(200,2))]
in_0_1_2 = [(5,0.1, pList(5,4))]
async_res = []
ids = []
for el in in_0_1:
    b_re = dview.apply_async(run_sim, *el)
    async_res.append(b_re)
    ids.extend(b_re.msg_ids)


In [36]:
rc.get_result(ids, owner=False).wait_interactive()

  12/12 tasks finished after   25 s
done


In [38]:
res = [r.get() for r in async_res]
len(res)

12

In [53]:
#res = [r.get() for r in async_res]
results = []
tmp = combine_results(*res)
results.append(tmp[0])
while len(tmp[1]):
    tmp = combine_results(*tmp[1])
    results.append(tmp[0])
results[0]

{'Assets': [array([1.01064575, 1.01065527, 1.00984695]),
  array([1.01064575, 1.01065527, 1.00984695]),
  array([1.01064575, 1.01065527, 1.00984695]),
  array([1.01064575, 1.01065527, 1.00984695]),
  array([1.01064575, 1.01065527, 1.00984695]),
  array([1.01064575, 1.01065527, 1.00984695])],
 'Assets var': [array([0.02303463, 0.02335925, 0.02316355]),
  array([0.02303463, 0.02335925, 0.02316355]),
  array([0.02303463, 0.02335925, 0.02316355]),
  array([0.02303463, 0.02335925, 0.02316355]),
  array([0.02303463, 0.02335925, 0.02316355]),
  array([0.02303463, 0.02335925, 0.02316355])],
 'Delta': [array([[ 2.90635629e-001,  7.09161041e-310,  7.08026658e-310],
         [ 5.51949636e-317,  2.93612046e-001,  5.51077462e-317],
         [ 7.91145769e+087, -2.67624541e-301,  2.91286018e-001],
         [ 7.19754365e-001,  7.09160877e-310,  7.08026658e-310],
         [ 5.51949636e-317,  7.16787464e-001,  5.51077462e-317],
         [-2.08999187e+260, -3.30277253e-101,  7.18305381e-001]]),
  array([

In [50]:
bb = combine_results(*aa[1])
bb[1]

[]

In [45]:
np.array(res[0]['Assets'])[:,:,0]

IndexError: too many indices for array